In [48]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import requests
import base64
import sqlite3 as db
sys.path.append('../')

pd.options.display.max_rows = 99

In [67]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [53]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [138]:
def getTrailheadRef():
    trailheadRef = pd.DataFrame()
    cnx = connectDB()
    try:
        trailheadRef = pd.read_sql('select * from trailheadRef;',con=cnx)
        trailheadRef.set_index('index',inplace=True)
        trailheadRef.convert_dtypes()
    except db.DatabaseError as dbError:
        print(dbError)
    cnx.close()
    return trailheadRef

In [140]:
trailheadRef = getTrailheadRef()

In [63]:
##  For the sake of demonstration please assume the data in the 'trailheadRef' comes from a King County Metro service
##  and/or is user-provided so some cleaning is required
def clean_trailheadRef(Latitude: float,Longitude: float, Address: str):
    cnx = connectDB()
    cur = cnx.cursor()
    trailheads = getTrailheadRef()
    ##set types for error checking, other cleaning
    trailheads.Latitude = trailheads['Latitude'].fillna(0.00).astype('float64',errors='ignore')
    trailheads.Longitude = trailheads['Longitude'].fillna(0.00).astype('float64',errors='ignore')
    trailheads.Address = trailheads['Address'].astype(str)

    updateMask = trailheads.apply(lambda x: bool(x.Address.strip()) and (x.Latitude == 0.00) and (x.Longitude == 0.00),axis=1)
    ##write stops that fail the updateMask checks to the db as logs
    try:
        trailheads[updateMask].copy().to_sql('geo_errorLogs',con=cnx,if_exists='append')
    except db.DatabaseError as dbError:
        cur.execute('INSERT INTO sql_errorLogs VALUES (?, ?)',dbError,str(dt.date.today()))
    cnx.close()
    return trailheads[~updateMask]
    

    ##needsGeo = trailheads[updateMask].copy()
    ##TODO: #25 fetch coordinates of trailheads in the update mask using Nominatim address service
    ##TODO #48 merge or call results of clean_trailheadRef() with dropCases()
#def dropCases():
    #dropCase = clean_trailheadRef()
    #dropMask = dropCase.apply(lambda y: (y.Address == '') | (y.Address == 'None'),axis=1)
    #noGeo = [y for y in list(dropCase[dropMask].StopName) if y!='']
    #if noGeo:
        #raise Exception('unable to fetch coordinates for {",".join(noGeo)}')
    #return dropCase[~dropMask]
    ##if there is no address or coordinates, drop the line


Trailhead eBird hotspot update

1. Ask the eBird API for the latest list of hotspots for each trailhead
2. Add new eBird hotspots to the table 'Hotspots' in the trailheadDirectBirds_sous database
3. Update hotspot data

In [126]:
def get_trailheadHotspots(radius: int = 4,fmt: str = 'json'):
    cleanTrailheads = pd.DataFrame(clean_trailheadRef(trailheadRef.Latitude,trailheadRef.Longitude,trailheadRef.Address))
    import time
    trailheadHotspots = []
    try:
        for StopName in cleanTrailheads.itertuples():
            time.sleep(0.3)
            ebird_url = 'https://api.ebird.org/v2/ref/hotspot/geo?'
            ebird_auth_header = {'X-eBirdApiToken': ebird_token}
            ebird_params = {
                'lat': str(StopName.Latitude),
                'lng': str(StopName.Longitude),
                'dist': str(radius),
                'fmt': str(fmt)
            }
            ebird_request = requests.get(ebird_url,headers=ebird_auth_header,params=ebird_params)
            ebird_response = pd.DataFrame(ebird_request.json())
            if ebird_request.status_code == requests.codes.ok:
                ebird_response['StopName'] = StopName.StopName
                trailheadHotspots.append(ebird_response)
            ebird_request.raise_for_status()
    except Exception as ee:
        raise UserWarning(ee)
    all_trailheadHotspots = pd.DataFrame()
    all_trailheadHotspots = pd.concat(trailheadHotspots,ignore_index=True)
    return all_trailheadHotspots

In [94]:
##hotspots = get_trailheadHotspots()

def update_trailheadHotspots():
    cnx = connectDB()
    new_hotspots = pd.DataFrame(get_trailheadHotspots())
    ##new_hotspots = pd.DataFrame(hotspots)
    new_hotspots['latestUpdate'] = dt.datetime.today()
    try:
        current_hotspots = pd.DataFrame()
        current_hotspots = pd.read_sql('select * from hotspots',cnx,parse_dates=["latestObsDt","latestUpdate"])
        current_hotspots.set_index('index',inplace=True)
    except Exception as ex:
        raise UserWarning(ex.__cause__)
    else:
        frames = [new_hotspots,current_hotspots]
        all_hotspots = pd.concat(frames,ignore_index=True)
        all_hotspots.sort_values(by=['StopName','locId','latestObsDt','latestUpdate'],ascending=False,inplace=True)
        all_hotspots.drop_duplicates(subset=['StopName','locId'],keep='first',inplace=True)
        all_hotspots.to_sql(name='Hotspots_cooking',con=cnx,if_exists='append')
        cnx.close()
    return all_hotspots